In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [2]:

# Directories
train_dir = "E:\CAPSTONE\Bird_Species_Classification\DataSet_V1_8020/train"
val_dir = "E:\CAPSTONE\Bird_Species_Classification\DataSet_V1_8020/val"
test_dir = "E:\CAPSTONE\Bird_Species_Classification\DataSet_V1_8020/test"



In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")
val_data = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode="categorical", shuffle=False)


Found 4434 images belonging to 14 classes.
Found 1115 images belonging to 14 classes.
Found 1393 images belonging to 14 classes.


In [7]:

# Number of classes
num_classes = train_data.num_classes

# Model setup
base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the pretrained layers

# Custom model
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Lower learning rate for stability
    metrics=["accuracy"]
)

# Callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint("ResNet50_BirdSpecies.keras", save_best_only=True)
]

# Training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,  # Start with 15 epochs and adjust based on results
    callbacks=callbacks
)



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 44s 0us/step


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.0642 - loss: 2.8272

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 1306s 9s/step - accuracy: 0.0643 - loss: 2.8263 - val_accuracy: 0.1049 - val_loss: 2.5677
Epoch 2/15
137/139 ━━━━━━━━━━━━━━━━━━━━ 39s 20s/step - accuracy: 0.1531 - loss: 2.5657

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.2f}")


In [ ]:

model.save("Model_V1_8020.h5")
